In [4]:
#!/usr/bin/env python3
import logging
from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

# Properties
props = Properties()
props["pegasus.data.configuration"] = "condorio"
props["dagman.retry"] = "5"
props.write()

#Replica
rc = ReplicaCatalog()\
        .add_replica("local", "saved_stated.txt", Path(".").resolve() / "saved_stated.txt")\
        .write()

# Transformation
check = Transformation(
        "print_nums1.py",
        site="local",
        pfn=str(Path(".").resolve() / "print_nums1.py"),
        is_stageable=True
    )

tc = TransformationCatalog().add_transformations(check).write()

# Workflow
job = Job(check)\
        .add_checkpoint(File("saved_stated.txt"), stage_out=True)\
        .set_stdout("output.txt")\
        .add_profiles(Namespace.PEGASUS, key="checkpoint.time", value=1)\
        .add_profiles(Namespace.PEGASUS, key="maxwalltime", value=2)


wf = Workflow("checkpoint-wf").add_jobs(job)

In [5]:
try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.20 21:22:13.634 UTC:    
2020.07.20 21:22:13.639 UTC:   ----------------------------------------------------------------------- 
2020.07.20 21:22:13.645 UTC:   File for submitting this DAG to HTCondor           : checkpoint-wf-0.dag.condor.sub 
2020.07.20 21:22:13.650 UTC:   Log of DAGMan debugging messages                 : checkpoint-wf-0.dag.dagman.out 
2020.07.20 21:22:13.656 UTC:   Log of HTCondor library output                

[##################################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/Workflow5/scitech/pegasus/checkpoint-wf/run0007
 Total jobs         :      9 (100.00%)
 # jobs succeeded   :      9 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total count 